Install requirements and download Spacy small

In [39]:
%pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement Tkinter (from versions: none)
ERROR: No matching distribution found for Tkinter


Import List

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

Download Spacy

In [ ]:
import spacy

# Define the model name
model_name = "en_core_web_sm"

# Check if the model is installed
if model_name not in spacy.util.get_installed_models():
    print(f"Downloading and installing the '{model_name}' model...")
    
    # Download and install the model
    !python -m spacy download {model_name}
else:
    print(f"The '{model_name}' model is already installed.")


Create Databases

In [ ]:
import os
import sqlite3

# Define the folder path where the databases will be stored
folder_path = "databases"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Database names
database_names = ["TrainData.db", "UnprocessedData.db"]

# Loop to create and set up the databases
for database_name in database_names:
    database_path = os.path.join(folder_path, database_name)

    # Check if the database file already exists
    if not os.path.exists(database_path):
        # Connect to the SQLite database (or create it if it doesn't exist)
        connection = sqlite3.connect(database_path)

        # Create a cursor object to execute SQL commands
        cursor = connection.cursor()

        # Create a table for each database (you can define the table schema as needed)
        cursor.execute('''CREATE TABLE conversations (
                            id INTEGER PRIMARY KEY AUTOINCREMENT,
                            user TEXT,
                            message TEXT
                         )''')

        # Commit the changes and close the connection
        connection.commit()
        connection.close()

        print(f"Database '{database_name}' created in the '{folder_path}' folder.")
    else:
        print(f"Database '{database_name}' already exists in the '{folder_path}' folder. Skipping creation.")

print("All databases created or skipped if they already exist.")


Takes all Data in the Data Folder and puts it in the Unprocessed Database

In [48]:
import sqlite3
import pyarrow.parquet as pq
from tqdm import tqdm

# Define the path to the Parquet file
parquet_file = "Data/data.parquet"

# Connect to the "UnprocessedData.db" database
db_connection = sqlite3.connect("databases/UnprocessedData.db")
cursor = db_connection.cursor()

# Read data from the Parquet file
parquet_data = pq.read_table(parquet_file)

# Check if the table already exists
table_exists_query = "SELECT name FROM sqlite_master WHERE type='table' AND name='text_data';"
cursor.execute(table_exists_query)
table_exists = cursor.fetchone()

if not table_exists:
    # If the table doesn't exist, create it dynamically based on the Parquet schema
    parquet_schema = parquet_data.schema
    column_names = [field.name for field in parquet_schema]
    column_definitions = ', '.join(['{} TEXT'.format(name) for name in column_names])
    create_table_query = f'CREATE TABLE IF NOT EXISTS text_data ({column_definitions})'
    cursor.execute(create_table_query)
    db_connection.commit()

# Use tqdm for the progress bar
with tqdm(total=parquet_data.num_rows, unit=" row", desc="Inserting Data") as pbar:
    # Iterate through the rows of the Parquet data and insert into the database
    for row in parquet_data:
        # Specify the correct column names
        id, system_prompt, question, response = row

        # Insert the row's values into the database
        insert_query = "INSERT INTO text_data (id, system_prompt, question, response) VALUES (?, ?, ?, ?)"
        cursor.execute(insert_query, (id, system_prompt, question, response))
        db_connection.commit()

        # Update the progress bar
        pbar.update(1)

# Close the database connection
db_connection.close()

print("Data from the Parquet file has been inserted into the 'UnprocessedData.db' database.")


Inserting Data:   0%|          | 0/3239027 [00:00<?, ? row/s]


ValueError: too many values to unpack (expected 4)

Preprocess Data

In [ ]:
import sqlite3
import spacy
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Function to perform text preprocessing
def preprocess_text(text, nlp):
    doc = nlp(text)
    tokens = [token.text for token in doc]

    tokens = [token.lower() for token in tokens]
    tokens = [token.translate(str.maketrans('', '', string.punctuation)) for token in tokens]

    stopwords_list = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stopwords_list]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    cleaned_text = ' '.join(tokens)

    return cleaned_text

# Function to create the "text_data" table in the database
def create_text_data_table(db_path):
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS text_data (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        content TEXT
                     )''')
    connection.commit()
    connection.close()

# Function to process and move data from UnprocessedData to TrainData
def process_and_move_data(unprocessed_db_path, train_db_path):
    create_text_data_table(train_db_path)  # Create the table if it doesn't exist

    # Connect to the "UnprocessedData.db" database
    unprocessed_db_connection = sqlite3.connect(unprocessed_db_path)
    unprocessed_cursor = unprocessed_db_connection.cursor()

    # Connect to the "TrainData.db" database
    train_db_connection = sqlite3.connect(train_db_path)
    train_cursor = train_db_connection.cursor()

    # Retrieve unprocessed data from the "UnprocessedData" database
    unprocessed_cursor.execute("SELECT id, content FROM text_data")
    unprocessed_rows = unprocessed_cursor.fetchall()

    # Create a spaCy NLP object for tokenization
    nlp = spacy.load("en_core_web_sm")

    # Process the data and insert into the "TrainData" database
    for row in unprocessed_rows:
        id, content = row
        preprocessed_text = preprocess_text(content, nlp)
        train_cursor.execute("INSERT INTO text_data (content) VALUES (?)", (preprocessed_text,))
        train_db_connection.commit()

    # Close the database connections
    unprocessed_db_connection.close()
    train_db_connection.close()

    print("Data processed and inserted into the 'TrainData.db' database.")

# Path to the "UnprocessedData.db" and "TrainData.db" databases
unprocessed_db_path = "databases/UnprocessedData.db"
train_db_path = "databases/TrainData.db"

# Call the function to process and move data
process_and_move_data(unprocessed_db_path, train_db_path)
